In [ ]:
# Import required libraries
import sys
sys.path.append('..')

from detection.yolo_detector import YOLODetector
import cv2
import matplotlib.pyplot as plt
from pathlib import Path
import numpy as np

print("✓ Libraries imported successfully")

## 1. Initialize YOLO Detector

We'll use the YOLOv8 nano model for fast detection.

In [ ]:
# Initialize detector
detector = YOLODetector(
    model_path="yolov8n.pt",
    confidence=0.5,
    device="cuda"  # Change to "cpu" if no GPU
)

print(f"Model loaded: {detector.model_path}")
print(f"Device: {detector.device}")
print(f"Total classes: {len(detector.class_names)}")

## 2. Load Test Image

Let's load a test image and see what objects we can detect.

In [ ]:
# Load image
image_path = "../data/input/test.jpg"
image = cv2.imread(image_path)

if image is None:
    print(f"Error: Could not load image from {image_path}")
else:
    h, w = image.shape[:2]
    print(f"Image loaded: {w}x{h}")
    
    # Display original image
    plt.figure(figsize=(12, 8))
    plt.imshow(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
    plt.title("Original Image")
    plt.axis('off')
    plt.show()

## 3. Detect Objects

Run YOLO detection on the image.

In [ ]:
# Detect objects
detections, inference_time = detector.detect(image)

print(f"Detections found: {len(detections)}")
print(f"Inference time: {inference_time*1000:.2f} ms")
print(f"FPS: {1/inference_time:.2f}")

print("\nDetected objects:")
for i, det in enumerate(detections, 1):
    print(f"{i}. {det['class_name']}: {det['confidence']:.2f}")

## 4. Visualize Results

Draw bounding boxes and labels on the image.

In [ ]:
# Draw detections
annotated = detector.draw_detections(image, detections)

# Display result
plt.figure(figsize=(15, 10))
plt.imshow(cv2.cvtColor(annotated, cv2.COLOR_BGR2RGB))
plt.title(f"YOLO Detection Results - {len(detections)} objects detected")
plt.axis('off')
plt.show()

# Save result
output_path = "../../results/images/notebook_yolo_result.jpg"
Path(output_path).parent.mkdir(parents=True, exist_ok=True)
cv2.imwrite(output_path, annotated)
print(f"Result saved to: {output_path}")

## 5. Detection Details

Let's examine each detection in detail.

In [ ]:
import pandas as pd

# Create DataFrame with detection details
det_data = []
for det in detections:
    x1, y1, x2, y2 = det['bbox']
    det_data.append({
        'Class': det['class_name'],
        'Confidence': f"{det['confidence']:.3f}",
        'Bbox': f"({x1}, {y1}, {x2}, {y2})",
        'Width': x2 - x1,
        'Height': y2 - y1,
        'Area': (x2 - x1) * (y2 - y1)
    })

df = pd.DataFrame(det_data)
display(df)

## 6. Detect Specific Classes

We can filter detections to only show specific object classes.

In [ ]:
# Detect only people (class_id=0)
people_detections, _ = detector.detect(image, classes=[0])

print(f"People detected: {len(people_detections)}")

# Visualize
if len(people_detections) > 0:
    people_annotated = detector.draw_detections(image, people_detections)
    
    plt.figure(figsize=(15, 10))
    plt.imshow(cv2.cvtColor(people_annotated, cv2.COLOR_BGR2RGB))
    plt.title(f"People Detection - {len(people_detections)} persons found")
    plt.axis('off')
    plt.show()

## 7. Benchmark Performance

Let's benchmark the detector's performance.

In [ ]:
import time

# Run multiple inferences
num_runs = 50
times = []

print(f"Running {num_runs} inference iterations...")

for i in range(num_runs):
    start = time.time()
    detections, _ = detector.detect(image)
    elapsed = time.time() - start
    times.append(elapsed)
    
    if (i + 1) % 10 == 0:
        print(f"  {i + 1}/{num_runs} completed")

# Calculate statistics
avg_time = np.mean(times) * 1000
std_time = np.std(times) * 1000
min_time = np.min(times) * 1000
max_time = np.max(times) * 1000
avg_fps = 1 / np.mean(times)

print("\n" + "="*50)
print("PERFORMANCE STATISTICS")
print("="*50)
print(f"Average time: {avg_time:.2f} ms (± {std_time:.2f} ms)")
print(f"Min time: {min_time:.2f} ms")
print(f"Max time: {max_time:.2f} ms")
print(f"Average FPS: {avg_fps:.2f}")
print("="*50)

## 8. Visualize Performance Distribution

Plot the distribution of inference times.

In [ ]:
# Plot inference time distribution
plt.figure(figsize=(12, 5))

plt.subplot(1, 2, 1)
plt.hist(np.array(times) * 1000, bins=20, edgecolor='black', alpha=0.7)
plt.xlabel('Inference Time (ms)')
plt.ylabel('Frequency')
plt.title('Inference Time Distribution')
plt.axvline(avg_time, color='red', linestyle='--', label=f'Mean: {avg_time:.2f} ms')
plt.legend()
plt.grid(True, alpha=0.3)

plt.subplot(1, 2, 2)
plt.plot(np.array(times) * 1000, linewidth=2)
plt.xlabel('Run Number')
plt.ylabel('Inference Time (ms)')
plt.title('Inference Time Over Runs')
plt.axhline(avg_time, color='red', linestyle='--', label=f'Mean: {avg_time:.2f} ms')
plt.legend()
plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## Summary

This notebook demonstrated:
- ✅ YOLO detector initialization
- ✅ Object detection in images
- ✅ Visualization with bounding boxes
- ✅ Class-specific filtering
- ✅ Performance benchmarking

**Next Steps:**
- Try with your own images
- Experiment with different confidence thresholds
- Test different YOLO models (yolov8s, yolov8m, yolov8l)